In [1]:
#Get YOLO from ultarlytics
!pip install ultralytics opencv-python-headless matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 16.5 MB/s eta 0:00:00a 0:00:01


In [2]:
import os   # to interact with operating system
import shutil   # for copying files and folders
import numpy as np  # to work with arrays
import pandas as pd  # to do data manupulationa and analysis
import itertools # to manupulate iterables
import glob # to search for files that match specific pattern

from sklearn.model_selection import train_test_split  # provides functions for splitting data into training and test sets, evaluating machine learning models, and performing cross-validation
from ultralytics import YOLO  # to get the yolo model

import wandb  # Weights and Biases ie. a machine learning development platform that allows users to track and visualize various aspects of their model training process in real-time
from kaggle_secrets import UserSecretsClient

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")
wandb.login(key=wandb_api_key)

data_dir = '/kaggle/input/militaryaircraftdetectiondataset/dataset'
output_dir = '/kaggle/working/'

merged_csv = os.path.join(output_dir, 'merged_annotations.csv')

all_files = glob.glob(os.path.join(data_dir, "*.csv"))

if not all_files:
    raise FileNotFoundError(f"No CSV files found in directory: {data_dir}")

df_list = []
for file in all_files:
    try:
        df = pd.read_csv(file)
        df_list.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

if df_list:
    merged_df = pd.concat(df_list, ignore_index=True)
    merged_df.to_csv(merged_csv, index=False)
else:
    raise ValueError("No dataframes to concatenate")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: susanthbondalapati (susanthbondalapati-ut-arlington-uta-the-university-of-te). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [7]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Object Detection using yolov8",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

acc,▁▆▆▇▆▇▆█
loss,█▄▃▄▄▃▂▁
acc,0.85967
loss,0.13511


In [9]:
class_names = ['MQ9', 'RQ4']  
num_classes = len(class_names)
class_id_map = {name: idx for idx, name in enumerate(class_names)}


def convert_to_yolo_format(row):
    dw = 1. / row['width']
    dh = 1. / row['height']
    x_center = (row['xmin'] + row['xmax']) / 2.0 * dw
    y_center = (row['ymin'] + row['ymax']) / 2.0 * dh
    width = (row['xmax'] - row['xmin']) * dw
    height = (row['ymax'] - row['ymin']) * dh
    return [class_id_map[row['class']], x_center, y_center, width, height]


train_df, val_df = train_test_split(merged_df, test_size=0.2, random_state=42)


def validate_and_clean_labels(df):
    valid_rows = []
    for _, row in df.iterrows():
        if row['class'] in class_id_map:
            valid_rows.append(row)
        else:
            print(f"Invalid class {row['class']} in {row['filename']}")
    return pd.DataFrame(valid_rows)

train_df = validate_and_clean_labels(train_df)
val_df = validate_and_clean_labels(val_df)

Invalid class C5 in f241ce08ce7aae4b879b0f9f59e8c097
Invalid class H6 in 094716a7e1680301904d3b5a9ade953f
Invalid class F4 in 0ab5313b67a25e5dd1d759a655c4ac46
Invalid class C2 in 203490476a18e1b50f4ac9f2db117e29
Invalid class F18 in d5e83b263e5c9015049fa18bbe7372bc
Invalid class F22 in a3eb324176a26a86f120e667e2aa1150
Invalid class F16 in 00465d490afac0ebd8a5775e72aedfca
Invalid class Rafale in 8a51f384a5d8ff7a0cdf88b707a5e7ae
Invalid class V22 in 07bdf6633833fcd4a7bf1390297aac02
Invalid class F16 in ffebf486be62b735ad87ab0ab5ef4498
Invalid class F18 in 171bce94adc9fe6297384ddd2212f764
Invalid class KC135 in c2c2d1f54f2d48dd430c9c547847c57e
Invalid class B2 in d22b380b05cc28556890b701082e4d14
Invalid class Su24 in 7a3a9af0fba259261a002cba765e295d
Invalid class C130 in 4db7b0cd0bfc979f940f620bea4d60a3
Invalid class WZ7 in 21597773fd5c4da817cf2ade1dbbec69
Invalid class EF2000 in 08a963a25210f023417e0a2e62bca32a
Invalid class J10 in e347ef4626e3b5fb6f0f5ada1c86a54d
Invalid class V22 in 40

In [10]:
def create_yolo_files(df, split):
    for _, row in df.iterrows():
        yolo_data = convert_to_yolo_format(row)
        txt_filename = os.path.join(output_dir, split, os.path.splitext(row['filename'])[0] + '.txt')
        img_filename = os.path.join(data_dir, row['filename'] + '.jpg')  
        output_img_filename = os.path.join(output_dir, split, row['filename'] + '.jpg')
        
        os.makedirs(os.path.dirname(txt_filename), exist_ok=True)
        
        with open(txt_filename, 'w') as f:
            f.write(' '.join(map(str, yolo_data)) + '\n')
        
     
        if os.path.exists(img_filename):
            shutil.copy(img_filename, output_img_filename)
        else:
            print(f"Warning: {img_filename} not found!")

In [12]:
create_yolo_files(train_df, 'train')
create_yolo_files(val_df, 'val')


model = YOLO('yolo11n.pt')  # importing model from here change it to yolov11 for next use


data_yaml = """
train: /kaggle/working/train
val: /kaggle/working/val
nc: {num_classes}
names: {class_names}
"""


data_yaml = data_yaml.format(num_classes=num_classes, class_names=class_names)
data_yaml_path = '/kaggle/working/data.yaml'
with open(data_yaml_path, 'w') as f:
    f.write(data_yaml)

100%|██████████| 5.35M/5.35M [00:00<00:00, 76.3MB/s]


In [14]:
training_config = {
    'data': data_yaml_path,
    'epochs': 100,
    'batch': 16,
    'imgsz': 640,
    'device': '0,1', 
    'project': 'AircraftDetectYolo',
    'name': 'UAV_training',
    'exist_ok': True,
    'workers': 4,
    'lr0': 0.002, 
}

In [15]:

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


model.train(**training_config)


metrics = model.val(data=data_yaml_path)

Ultralytics 8.3.75 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
                                                       CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/working/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0,1, workers=4, project=AircraftDetectYolov8, name=UAV_training, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fram

100%|██████████| 755k/755k [00:00<00:00, 18.1MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

val: Scanning /kaggle/working/val.cache... 110 images, 0 backgrounds, 0 corrupt: 100%|██████████| 110/110 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  3.20it/s]


                   all        110        110      0.762      0.667       0.72      0.553
                   MQ9         60         60      0.835      0.759      0.813       0.64
                   RQ4         50         50      0.688      0.575      0.627      0.465


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 9.7ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to AircraftDetectYolov8/UAV_training
